# DreamerV3 for Model-Based RL in OpenScope

This notebook demonstrates a simplified DreamerV3-style workflow: learn a latent world model and train a policy from imagined trajectories for sample-efficient RL.

## Why DreamerV3?
- Model-based RL with imagination enables much higher sample efficiency
- Joint training of world model, actor, and critic
- Strong empirical results across control tasks

## Workflow
1. Load/collect offline trajectories from OpenScope
2. Train world model (encoder + RSSM + decoders)
3. Train actor/critic with imagined rollouts
4. Evaluate policy in OpenScope

## Prerequisites
- Offline dataset (or OpenScope server for collection)
- GPU recommended



## Note on External Library

This demo uses an external implementation of DreamerV3 (TorchRL) to keep the notebook focused on OpenScope integration rather than re-implementing the algorithm. In production, you can pin exact versions and move the config into code.

- Library: TorchRL (DreamerV3 implementation)
- Install: see the next cell for a one-time install command

### Production Config (recommended starting point)
- World model: latent_dim=64, rssm_hidden=200, rssm_stochastic=32, rssm_discrete=False
- Imagination: horizon=15–50, gamma=0.997
- Optimization: actor_lr=3e-4, critic_lr=3e-4, model_lr=3e-4, batch_size=512
- Replay: capacity=1e6 transitions, prefill=5e4 random steps
- Training cadence: update_every=16 env steps, updates_per_step=1.0
- Parallelism: 8–16 envs, collector on separate process
- Logging: WandB/TensorBoard, checkpoints every 10k updates

## Summary

- This notebook outlines a DreamerV3-style training flow using TorchRL.
- For production, switch to the provided production config and use TorchRL collectors + trainers.
- Integrate the policy with `PlaywrightEnv` for evaluation once trained.


In [ ]:
# Optional: Install TorchRL with DreamerV3 implementation
# Uncomment if not already installed in your environment
# %pip install torchrl==0.5.0 torch>=2.2.0 torchvision --extra-index-url https://download.pytorch.org/whl/cu121

import sys
from pathlib import Path
import numpy as np
import torch

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

from environment.utils import get_device

print("✅ Environment ready")
print(f"PyTorch: {torch.__version__}")
print(f"Device: {get_device()}")


## Section 1: Load or Collect Trajectories

We will use the same offline data format as other notebooks. If none found, collect a small set via Playwright (slower).


In [ ]:
from pathlib import Path
from data.offline_dataset import OfflineDatasetCollector

# Try loading a pre-collected dataset
DATA_PATH = Path("../data/offline_data.pkl")

if DATA_PATH.exists():
    print(f"📦 Loading episodes from {DATA_PATH}")
    episodes = OfflineDatasetCollector.load_episodes(str(DATA_PATH))
else:
    print("⚠️ No dataset at ../data/offline_data.pkl. You can collect a small set below (slower).")
    episodes = []

print(f"Episodes: {len(episodes)}")


In [ ]:
# Optional: small data collection via Playwright (random policy)
from environment import PlaywrightEnv

if len(episodes) == 0:
    try:
        print("🎬 Collecting 20 episodes (demo)")
        env = PlaywrightEnv(headless=True, timewarp=5, max_aircraft=5, episode_length=300)
        collector = OfflineDatasetCollector(env)
        episodes = collector.collect_random_episodes(num_episodes=20, max_steps=100, verbose=True)
        print(f"Collected {len(episodes)} episodes")
    except Exception as e:
        print(f"Data collection failed: {e}")


## Section 2: Build Replay Buffer (TorchRL format)

Convert the episodes into a replay buffer of (s, a, r, s', done).


In [ ]:
from torchrl.data import ReplayBuffer, ListStorage

if len(episodes) == 0:
    raise RuntimeError("No episodes available. Please load or collect data above.")

# Build a simple replay buffer
rb = ReplayBuffer(storage=ListStorage())

count = 0
for ep in episodes:
    for t in range(ep.length - 1):
        s = ep.observations[t]
        a = ep.actions[t]
        r = ep.rewards[t]
        s_next = ep.observations[t+1]
        d = ep.dones[t]
        rb.add({
            'state': s,
            'action': a,
            'reward': r,
            'next_state': s_next,
            'done': d,
        })
        count += 1

print(f"✅ ReplayBuffer filled with {count} transitions")


## Section 3: Configure DreamerV3 (Demo vs Production)

We define a small demo config for quick runs and a production config you can switch to.


In [ ]:
demo_cfg = {
    'latent_dim': 32,
    'rssm_hidden': 128,
    'rssm_stochastic': 16,
    'rssm_discrete': False,
    'imagination_horizon': 15,
    'gamma': 0.997,
    'actor_lr': 3e-4,
    'critic_lr': 3e-4,
    'model_lr': 3e-4,
    'batch_size': 256,
}

prod_cfg = {
    'latent_dim': 64,
    'rssm_hidden': 200,
    'rssm_stochastic': 32,
    'rssm_discrete': False,
    'imagination_horizon': 30,
    'gamma': 0.997,
    'actor_lr': 3e-4,
    'critic_lr': 3e-4,
    'model_lr': 3e-4,
    'batch_size': 512,
}

cfg = demo_cfg
print("Using DreamerV3 config:", cfg)


## Section 4: DreamerV3 Setup (TorchRL)

Set up DreamerV3 components. This is a minimal sketch; see TorchRL docs for advanced usage.


In [ ]:
# Minimal DreamerV3 setup with TorchRL (pseudo-API for clarity)
from torchrl.objectives import DreamerV3Loss
from torchrl.modules import RSSM

# Build a small RSSM world model (encoder/decoder omitted for brevity)
rssm = RSSM(
    hidden_dim=cfg['rssm_hidden'],
    stochastic_dim=cfg['rssm_stochastic'],
    discrete=False,
)

loss_module = DreamerV3Loss(
    rssm=rssm,
    actor_lr=cfg['actor_lr'],
    critic_lr=cfg['critic_lr'],
    model_lr=cfg['model_lr'],
    gamma=cfg['gamma'],
    imagination_horizon=cfg['imagination_horizon'],
)

print("✅ DreamerV3 components created (demo)")


## Section 5: Train World Model from Offline Data

Train the RSSM/world model on replay buffer (state transitions).


In [ ]:
from tqdm import trange

num_epochs = 5  # demo; increase for real training
batch_size = cfg['batch_size']

print("🚀 Training world model (demo)…")
for epoch in range(num_epochs):
    epoch_loss = 0.0
    # Simple manual batching from replay buffer storage
    storage = rb._storage._storage  # ListStorage
    np.random.shuffle(storage)
    for i in trange(0, len(storage), batch_size, leave=False):
        batch = storage[i:i+batch_size]
        # Convert batch to tensors and compute DreamerV3 losses (model/actor/critic)
        # NOTE: For brevity, this is a placeholder. Use TorchRL DataSpec + collectors in production.
        # loss = loss_module(batch)
        # loss.backward(); optimizer.step(); optimizer.zero_grad()
        pass
    print(f"Epoch {epoch+1}/{num_epochs} done")

print("✅ World model training (demo) complete")


## Section 6: Imagined Rollouts and Policy Training (Sketch)

Use the learned world model to generate imagined trajectories and update actor/critic.


In [ ]:
# Pseudo-code: imagined rollouts and policy updates
# for update in range(1000):
#     posterior = rssm.observe(replay_batch)
#     imagined_latents = rssm.imagine(posterior, horizon=cfg['imagination_horizon'])
#     actor_loss, critic_loss = loss_module.actor_critic_losses(imagined_latents)
#     (actor_loss + critic_loss).backward()
#     actor_opt.step(); critic_opt.step(); actor_opt.zero_grad(); critic_opt.zero_grad()

print("ℹ️ Imagined rollouts & policy training sketched. See TorchRL DreamerV3 examples for full training loops.")


## Section 7: Evaluation in OpenScope (Outline)

After training, evaluate the policy in `PlaywrightEnv`:

```python
from environment import PlaywrightEnv

env = PlaywrightEnv(headless=True, timewarp=5, max_aircraft=5, episode_length=600)
obs, info = env.reset()

total_reward = 0
for t in range(120):  # ~10 minutes at 5s interval
    # Use actor to sample action given latent state (pseudo)
    # action = actor.sample(obs)
    # For demo, sample random:
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    total_reward += reward
    if terminated or truncated:
        break

print("Episode reward:", total_reward)
```

Replace the random action with the DreamerV3 actor once trained end-to-end.



## References and Version Pinning

- TorchRL DreamerV3 tutorial: https://pytorch.org/torchrl/stable/tutorials/foundations/dreamer_v3.html
- Paper: DreamerV3 (Hafner et al.)

Recommended pins (example):

```bash
pip install "torch>=2.2,<2.4" torchvision \
  torchrl==0.5.0 tensordict==0.5.0 \
  --extra-index-url https://download.pytorch.org/whl/cu121
```

Switch `cfg = prod_cfg` above for production training.